# How to run code using Node.js Fibers
https://stackoverflow.com/questions/14854346/how-to-run-code-using-node-js-fibers

A fiber is a sort of lightweight thread of execution. Like real threads and processes, a fiber must be given a block of code to execute upon run.

In [1]:
var Fiber = require('fibers');

function sample(callback) {
    callback("this callback");
}
var f = Fiber(function() {
    var fiber = Fiber.current;

    sample(function(str) {
        fiber.run(str);
    });

    var str = Fiber.yield();
    console.log(str);
});

f.run();

Javascript Error: This Fiber is already running

上面的例子会给出Error: This Fiber is already running，原因是执行过程如下：
1. Set variable f as a fiber.
1. Run the fiber:
 1. Set variable fiber pointing to the current running fiber.
 1. Call function sample.
 1. Call the callback.
 1. Call fiber.run, which gives the error as the current fiber is already running.
 
上面的代码结构是没有问题的，问题在于它假定sample是个异步函数。下面把sample重新改写一下：

In [2]:
function sample(callback) {
    setTimeout(function() {
        callback("this callback");
    }, 500);
}
f.run();

改写过的执行过程：
1. 设置变量fiber指向当前运行的Ffiber
1. 调用函数sample()。该函数立即返回，暂时不调用callback
1. 调用Fiber.yield(), 暂停当前的fiber.
1. 500 ms后, 调用callback.
1. 调用fiber.run() 参数字符串'this callback', 恢复运行fiber.
1. Fiber.yield()返回，设置str to 'this callback'.
1. Log the string to console.

注意：step 4在fiber之外运行的

### The function of yield and run
Fiber必须协作地把控制交给另外的fiber，或者主执行程序。放弃控制权，需要调用：Fiber.yield()

而继续执行（直接从yield之后的那个点），必须要调用run()

把值传入传出fiber的机制，主要靠yield和run的互相配合。
- An argument given to run (which is outside the fiber), is returned by yield (inside the fiber).
- An argument given to yield (inside the fiber), is returned by run (outside the fiber).

In [3]:
//var Fiber = require('fibers');

Fiber(function() {
    let sleep = function (ms) {
        let fiber = Fiber.current;
        //console.log('enter sleep(), fiber = ' + fiber.toString());
        setTimeout(function() {
            //console.log('enter timeout callback');
            fiber.run();
        }, ms);
        Fiber.yield();
    };

    console.log('wait... ' + new Date);
    sleep(1000);
    console.log('ok... ' + new Date);
}).run();
console.log('back in main');

wait... Tue Dec 05 2017 17:46:09 GMT+0800 (CST)
back in main
this callback
ok... Tue Dec 05 2017 17:46:10 GMT+0800 (CST)
